In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler


# 파일 읽기
input_file = "C:\\Users\\kdecs\\Desktop\\VSCode\\dekmidterms\\6_shopping.csv"
# 대부분 파일은 엔코딩 cp949, utf-8을 씀
def read_file(file_path):
    df = pd.read_csv(file_path)
    df_original = df.copy() # 데이터 원본 보관
    return df, df_original

df, df_original = read_file(input_file)

# 데이터프레임 분석 및 확인
df.info()
df.describe()
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [29]:
#데이터 중복치 제거
def remove_dup(df):
    # 열 기준 중복치 제거
    df = df.drop_duplicates()
    return df

df = remove_dup(df)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [30]:
# 데이터 결측치 개수 확인 - 딕셔너리를 만들어 각 콜럼과 결측치 개수를 맵핑핑
def count_nan(df):
    count_nan_dict = {}
    for col in df:
        count_nan_dict[col] = df[col].isnull().sum()
    return count_nan_dict

nan_dict = count_nan(df)
print(nan_dict)


{'Invoice': np.int64(0), 'StockCode': np.int64(0), 'Description': np.int64(4275), 'Quantity': np.int64(0), 'InvoiceDate': np.int64(0), 'Price': np.int64(0), 'Customer ID': np.int64(235151), 'Country': np.int64(0)}


In [31]:
# 문자 결측치를 NaN으로 처리하는 코드
def change_nan(df):
    df.replace(['ERROR', 'UNKNOWN', '-', 'N/A', '?'], np.nan, inplace=True)
    return df

df_nodup = change_nan(df)

df_nodup.tail()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
1067370,581587,POST,POSTAGE,1,2011-12-09 12:50:00,18.00,12680.0,France


In [52]:
# 결측치가 너무 많은 행 & 모든 값이 같은 행, 그리고 모든 값이 다른 행 (ID등) 제거
def remove_nan_col(df):
    col_remove = []

    for col in df:
        emp_row = df[col].isna().sum() # 행의 결측치 개수
        total_row = df.shape[0] 
        unique_no = df[col].nunique() # 행의 고유값
        print(col, (emp_row/total_row))
        print(unique_no)
        # 결측치 개수가 총 행의 50%가 넘으면 제거 --> 유의미한 데이터 추출 어려움
        if ((emp_row / total_row) * 100) >= 50:
            col_remove.append(col)
        # 행의 모든 값이 똑같은 경우도 제거
        elif unique_no == 1:
            col_remove.append(col)
        # 행의 모든 값이 다를 경우도 제거 --> ID와 같은 값들임
        elif ((unique_no/total_row) * 100) >= 99:
            col_remove.append(col)
    df_filtered = df.drop(columns=col_remove)
    df_filtered = df.drop(['Customer ID', 'InvoiceDate'], axis=1)
    df_maintain = df[['Customer ID', 'InvoiceDate']]
    # 제거된 행들의 이름을 담은 리스트도 리턴
    return df_filtered, col_remove, df_maintain

df, col_remove, df_maintain = remove_nan_col(df_nodup)
#print(col_remove)
df_maintain.head()
# df. columns


Invoice 0.0
53628
StockCode 0.0
5305
Description 0.004225409375859118
5697
Quantity 0.0
1057
InvoiceDate 0.0
47635
Price 0.0
2807
Customer ID 0.22763098285054925
5942
Country 0.0
43


,Customer ID,InvoiceDate
0,13085.0,2009-12-01 07:45:00
1,13085.0,2009-12-01 07:45:00
2,13085.0,2009-12-01 07:45:00
3,13085.0,2009-12-01 07:45:00
4,13085.0,2009-12-01 07:45:00


In [53]:
# 행 분류 (연속형, 범주형) & 데이터타입 변환
def num_cat_col_classifier(df, threshold):
    num_col_list = []  # 연속형
    cat_col_list = []  # 범주형 

    for col in df.columns:
        no_unique = df[col].nunique()
        print(col, no_unique)
        # 행 이름에 날짜가 들어가면 날짜값으로 변환 시도 + 연속형 행에 추가
        if "date" in col.lower():
                df[col] = pd.to_datetime(df[col], errors='raise')
                # 날짜값을 인코딩시 부정확한 데이터가 생성되어 1970-01-01을 기준으로 일수로 변환
                # 날짜값 사이의 기간을 유의미한 데이터로 변환 가능
                # df[col] = (df[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
                num_col_list.append(col)  
        #고유값이 지정한 개수보다 많으면 연속형 범주로 포함 --> 데이터를 보고 threshold 설정
        elif no_unique >= threshold:
            try:
                # 숫자인데 데이터타입이 float가 아닌 경우 숫자로 변환
                df[col] = pd.to_numeric(df[col], errors='raise') 
                num_col_list.append(col)
            except Exception:
                # 숫자 변환에 에러 발생 시 멈추고 범주형 데이터에 추가
                cat_col_list.append(col)
        else:
            cat_col_list.append(col)

    return num_col_list, cat_col_list

num_list, cat_list = num_cat_col_classifier(df, 4)
num_df = df[num_list]
cat_df = df[cat_list]

print(num_df.head())
num_df.info()

print(cat_df.head())

print(num_df.dtypes)
print(cat_df.dtypes)



Invoice 53628
StockCode 5305
Description 5697
Quantity 1057
Price 2807
Country 43
   Quantity  Price
0        12   6.95
1        12   6.75
2        12   6.75
3        48   2.10
4        24   1.25
<class 'pandas.core.frame.DataFrame'>
Index: 1033036 entries, 0 to 1067370
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   Quantity  1033036 non-null  int64  
 1   Price     1033036 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 23.6 MB
  Invoice StockCode                          Description         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE   United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX  United Kingdom
Quantity      int64
Price       float64
dtyp

In [54]:
le = LabelEncoder()

#범주형 데이터 인코딩 함수
def label_cat_col(df):
    for col in df:
        df.loc[:, col] = df[col].fillna(df[col].mode()[0])
        # 고유값이 지정한 값보다 많은 경우 labelencoder 실행 --> 차원의 저주 방지
        if df[col].nunique() >= 5:
            df.loc[:, col] = le.fit_transform(df[col])
        # 고유값의 개수가 적으면 onehot encoding 실행 
        else:
            dummies = pd.get_dummies(df[col], prefix=col).astype(int)
            df = pd.concat([df.drop(columns=[col]), dummies], axis=1)
    return df

cat_df = label_cat_col(cat_df)
print(cat_df.head())
print(cat_df)
print(num_df.isna().sum())



  Invoice StockCode Description Country
0       0      4551          65      40
1       0      3701        3430      40
2       0      3703          36      40
3       0      1396        3764      40
4       0       711        4815      40
        Invoice StockCode Description Country
0             0      4551          65      40
1             0      3701        3430      40
2             0      3703          36      40
3             0      1396        3764      40
4             0       711        4815      40
...         ...       ...         ...     ...
1067366   45329      2215        1024      14
1067367   45329      2557        1033      14
1067368   45329      2558        1032      14
1067369   45329      1493         411      14
1067370   45329      5290        3688      14

[1033036 rows x 4 columns]
Quantity    0
Price       0
dtype: int64


In [55]:
# 이상치 탐지 & 제거 - IQR 방식
def check_outlier(df, threshold=1.5):
    for col in df:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        # IQR 값으로 생성한 상한선과 하한선을 데이터에 clip으로 적용
        df[col].clip(lower_bound, upper_bound)
    return df

num_df = check_outlier(num_df)
print(num_df.head())


   Quantity  Price
0        12   6.95
1        12   6.75
2        12   6.75
3        48   2.10
4        24   1.25


In [56]:
# 데이터 형태에 따라 정규화 혹은 표준화 적용
# 연속형 데이터 정규화
def num_normalized(df):
    for col in df:
        mms = MinMaxScaler()
        df.loc[:, col] = mms.fit_transform(df[[col]])
    return df

normal_num_df = num_normalized(num_df)
print(normal_num_df)


# 연속형 데이터 표준화
"""
def num_standardized(df):
    for col in df:
        ss = StandardScaler()
        df.loc[:, col] = ss.fit_transform(df[[col]])
    return df
standard_num_df = num_standardized(num_df)
print(standard_num_df)
"""

         Quantity     Price
0        0.500074  0.579071
1        0.500074  0.579069
2        0.500074  0.579069
3        0.500296  0.579018
4        0.500148  0.579009
...           ...       ...
1067366  0.500037  0.579018
1067367  0.500025  0.579040
1067368  0.500025  0.579040
1067369  0.500019  0.579049
1067370  0.500006  0.579190

[1033036 rows x 2 columns]


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29612\2279002822.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.50007408 0.50007408 0.50007408 ... 0.50002469 0.50001852 0.50000617]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = mms.fit_transform(df[[col]])


'\ndef num_standardized(df):\n    for col in df:\n        ss = StandardScaler()\n        df.loc[:, col] = ss.fit_transform(df[[col]])\n    return df\nstandard_num_df = num_standardized(num_df)\nprint(standard_num_df)\n'

In [65]:
# 범주형과 연속형 결합하기
def add_dfs(df1, df2, df_maintain):
    final_df = pd.concat([df1, df2, df_maintain], axis=1)
    return final_df

final_df = add_dfs(normal_num_df, cat_df, df_maintain)
final_df.head()


,Quantity,Price,Invoice,StockCode,Description,Country,Customer ID,InvoiceDate
0,0.500074,0.579071,0,4551,65,40,13085.0,2009-12-01 07:45:00
1,0.500074,0.579069,0,3701,3430,40,13085.0,2009-12-01 07:45:00
2,0.500074,0.579069,0,3703,36,40,13085.0,2009-12-01 07:45:00
3,0.500296,0.579018,0,1396,3764,40,13085.0,2009-12-01 07:45:00
4,0.500148,0.579009,0,711,4815,40,13085.0,2009-12-01 07:45:00


In [66]:
# CSV파일로 저장
def save_csv(df):
    # final_df = df.to_csv(f'output_file.csv', index=False)
    return final_df

save_csv(final_df)


,Quantity,Price,Invoice,StockCode,Description,Country,Customer ID,InvoiceDate
0,0.500074,0.579071,0,4551,65,40,13085.0,2009-12-01 07:45:00
1,0.500074,0.579069,0,3701,3430,40,13085.0,2009-12-01 07:45:00
2,0.500074,0.579069,0,3703,36,40,13085.0,2009-12-01 07:45:00
3,0.500296,0.579018,0,1396,3764,40,13085.0,2009-12-01 07:45:00
4,0.500148,0.579009,0,711,4815,40,13085.0,2009-12-01 07:45:00
...,...,...,...,...,...,...,...,...
1067366,0.500037,0.579018,45329,2215,1024,14,12680.0,2011-12-09 12:50:00
1067367,0.500025,0.579040,45329,2557,1033,14,12680.0,2011-12-09 12:50:00
1067368,0.500025,0.579040,45329,2558,1032,14,12680.0,2011-12-09 12:50:00
1067369,0.500019,0.579049,45329,1493,411,14,12680.0,2011-12-09 12:50:00


In [67]:
# 위 모든 함수를 실행하는 함수 생성
def some_function(input_file):
    df, df_original = read_file(input_file)
    df = remove_dup(df)
    df_nodup = change_nan(df)
    df, col_remove, df_maintain = remove_nan_col(df_nodup)
    num_list, cat_list = num_cat_col_classifier(df, 4)
    num_df = df[num_list]
    cat_df = df[cat_list]
    cat_df = label_cat_col(cat_df)
    num_df = check_outlier(num_df)
    normal_num_df = num_normalized(num_df)
    # standard_num_df = num_standardized(num_df)
    final_df = add_dfs(normal_num_df, cat_df, df_maintain)
    output_file = save_csv(final_df)
    return output_file

output_file = some_function(input_file)
output_file.head()

Invoice 0.0
53628
StockCode 0.0
5305
Description 0.004225409375859118
5697
Quantity 0.0
1057
InvoiceDate 0.0
47635
Price 0.0
2807
Customer ID 0.22763098285054925
5942
Country 0.0
43
Invoice 53628
StockCode 5305
Description 5697
Quantity 1057
Price 2807
Country 43


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29612\2279002822.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.50007408 0.50007408 0.50007408 ... 0.50002469 0.50001852 0.50000617]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = mms.fit_transform(df[[col]])


,Quantity,Price,Invoice,StockCode,Description,Country,Customer ID,InvoiceDate
0,0.500074,0.579071,0,4551,65,40,13085.0,2009-12-01 07:45:00
1,0.500074,0.579069,0,3701,3430,40,13085.0,2009-12-01 07:45:00
2,0.500074,0.579069,0,3703,36,40,13085.0,2009-12-01 07:45:00
3,0.500296,0.579018,0,1396,3764,40,13085.0,2009-12-01 07:45:00
4,0.500148,0.579009,0,711,4815,40,13085.0,2009-12-01 07:45:00


In [51]:
input_file = "C:\\Users\\kdecs\\Desktop\\VSCode\\dekmidterms\\6_shopping.csv"
output_file = some_function(input_file)
output_file.head()

Invoice 0.0
53628
StockCode 0.0
5305
Description 0.004225409375859118
5697
Quantity 0.0
1057
InvoiceDate 0.0
47635
Price 0.0
2807
Customer ID 0.22763098285054925
5942
Country 0.0
43
Invoice 53628
StockCode 5305
Description 5697
Quantity 1057
Price 2807
Country 43


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_29612\2279002822.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.50007408 0.50007408 0.50007408 ... 0.50002469 0.50001852 0.50000617]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = mms.fit_transform(df[[col]])


,Quantity,Price,Invoice,StockCode,Description,Country
0,0.500074,0.579071,0,4551,65,40
1,0.500074,0.579069,0,3701,3430,40
2,0.500074,0.579069,0,3703,36,40
3,0.500296,0.579018,0,1396,3764,40
4,0.500148,0.579009,0,711,4815,40


In [ ]:
"""
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('label_encoder', LabelEncoder())
])
col_transfomer = ColumnTransformer(transformers = [
    ('num_pipeline', num_pipeline, num_col_classifier(df)),
    ('cat_pipeline', cat_pipeline, cat_col_classifier(df))
])

steps = [
    ('remove_dup', FunctionTransformer(remove_dup, validate=True)),
    ('count_nan', FunctionTransformer(count_nan, validate=True)),
    ('remove_nan_col', FunctionTransformer(remove_nan_col, validate=True)),
    ('col_classifier', FunctionTransformer(num_col_classifier, validate=True)),
    ('col_classifier', FunctionTransformer(cat_col_classifier, validate=True)),
    ('column_transform', col_transfomer)
]

pipe_final = Pipeline(steps)
preprocessed_df = pipe_final.fit_transform(df)

output_file = 'preprocessed_data.csv'
preprocessed_df.to_csv(output_file, index=False)

print("Preprocessed DataFrame:")
print(preprocessed_df)
print(f"\nSaved to {output_file}")
"""

'\nnum_pipeline = Pipeline(steps=[\n    (\'impute\', SimpleImputer(strategy=\'median\')),\n    (\'scale\', StandardScaler())\n])\n\ncat_pipeline = Pipeline(steps=[\n    (\'impute\', SimpleImputer(strategy=\'most_frequent\')),\n    (\'label_encoder\', LabelEncoder())\n])\ncol_transfomer = ColumnTransformer(transformers = [\n    (\'num_pipeline\', num_pipeline, num_col_classifier(df)),\n    (\'cat_pipeline\', cat_pipeline, cat_col_classifier(df))\n])\n\nsteps = [\n    (\'remove_dup\', FunctionTransformer(remove_dup, validate=True)),\n    (\'count_nan\', FunctionTransformer(count_nan, validate=True)),\n    (\'remove_nan_col\', FunctionTransformer(remove_nan_col, validate=True)),\n    (\'col_classifier\', FunctionTransformer(num_col_classifier, validate=True)),\n    (\'col_classifier\', FunctionTransformer(cat_col_classifier, validate=True)),\n    (\'column_transform\', col_transfomer)\n]\n\npipe_final = Pipeline(steps)\npreprocessed_df = pipe_final.fit_transform(df)\n\noutput_file = \'pre